# **IE–IATA Datathon 2025** 
## EU27 Sustainable Aviation Fuel (SAF) Scenario Model  
**Team: The Incredibles**

This notebook implements an end-to-end, transparent modelling workflow to:

1. Reconstruct **historical EU27 aviation fuel demand** from Eurostat.
2. Integrate **EUROCONTROL** datasets on CO₂ emissions, flights, market segments, and flight phases.
3. Build a **calibration table (1990–2050)** combining fuel, emissions, activity, and operational data.
4. Construct two SAF pathways:
   - **Scenario 0 (S0)** – Business-as-Usual (BAU)
   - **Scenario 1 (S1)** – Policy / ReFuelEU-aligned
5. Compute:
   - CO₂ emissions for each scenario
   - Avoided emissions vs a fossil-only counterfactual
6. Add:
   - A **probability-weighted trajectory** reflecting IEA’s warning that BAU/high-warming pathways are more likely  
   - A **cost & sensitivity analysis** on:
     - SAF price premium (€/t)
     - ETS carbon price (€/tCO₂)
     - SAF lifecycle emission reduction (LCA %)

Final outputs:

- `ie_iata_output_dataset.csv` — official datathon deliverable  
- `ie_iata_prob_weighted_results.csv` — probability-weighted emissions  
- `ie_iata_sensitivity_results.csv` — economic sensitivity analysis  
- `eu27_calibration_table_1990_2050.csv` — internal calibration table  

---

## 📚 **SOURCES (OFFICIAL, PUBLIC, CREDIBLE)**

### **1. Fuel demand, flight activity & emissions**

These are the primary inputs for historical calibration and traffic-related modelling.

- **Eurostat – Final energy consumption: Air transport (EU27)**  
  https://ec.europa.eu/eurostat  
  *Used to reconstruct fuel consumption for 1990–2022.*

- **EUROCONTROL – Aviation Outlook 2050**  
  https://www.eurocontrol.int/publication/aviation-outlook-2050  
  *Used to justify the +1.6% annual growth rate and long-term fuel demand projection.*

- **EUROCONTROL – G2G Emissions Dataset**  
  https://ansperformance.eu/environment/emissions/  
  *Used for CO₂ by flight phase, market segment, and NB_FLIGHTS totals.*

- **EUROCONTROL – CO₂ by State Dashboard**  
  https://ansperformance.eu/environment/co2/  
  *Used to derive EU27 aggregated CO₂ & flight totals.*

---

### **2. SAF cost, price premium & economics**

These sources support sensitivity assumptions around SAF cost spread (€/t) vs fossil jet fuel.

- **IATA – Sustainable Aviation Fuel (SAF) Key Facts**  
  https://www.iata.org/en/programs/environment/saf/  
  *States SAF is still “significantly more expensive” than fossil jet fuel (typically 2–5×).*

- **ICAO CORSIA – Eligible Fuels & Emissions Factors**  
  https://www.icao.int/environmental-protection/CORSIA/Pages/default.aspx  
  *Provides certified LCA values and cost ranges for each fuel pathway.*

- **ICCT (International Council on Clean Transportation) – SAF Cost & Potential**  
  https://theicct.org/publication/aviation-global-saf-cost-jan2022/  
  *Shows:  
  - HEFA SAF ≈ 2–4× fossil jet  
  - FT-SPK ≈ 3–5×  
  - PtL e-fuels ≈ 5–8× (early stage)*

- **IATA Fuel Price Monitor** (for fossil jet reference price)  
  https://www.iata.org/en/publications/economics/fuel-monitor/

---

### **3. SAF lifecycle emission reduction (LCA performance)**

These support the SAF LCA sensitivity values: **60%, 75%, 90%**.

- **IATA – Net Zero Roadmap**  
  https://www.iata.org/en/programs/environment/net-zero/  
  *Typical SAF LCA reduction = 60–80%.*

- **EASA – ReFuelEU Technical Report**  
  https://www.easa.europa.eu/en/document-library/general-publications/refueleu-aviation-annual-technical-report  
  *Uses 70–80% LCA reduction in modelling.*

- **ICAO CORSIA – LCA Methodology**  
  https://www.icao.int/environmental-protection/CORSIA/Pages/CORSIA-Eligible-Fuels.aspx  
  *Pathways:  
  - HEFA: 60–80%  
  - FT-SPK: 80–90%  
  - PtL: 90%+*

- **Argonne National Laboratory – GREET Model**  
  https://greet.es.anl.gov/  
  *Peer-reviewed LCA tool widely used for aviation fuels.*

---

### **4. ETS carbon price ranges & policy**

These support the ETS sensitivity values: **50, 100, 150 €/tCO₂**.

- **European Commission – EU ETS (Phase IV) Reform / Fit for 55**  
  https://climate.ec.europa.eu/eu-action/eu-emissions-trading-system-eu-ets_en  
  *ETS prices expected in the 80–150 €/tCO₂ range by 2030–2035.*

- **EU ETS Factsheet (Official)**  
  https://climate.ec.europa.eu/system/files/2022-04/EU_ETS_Factsheet_en.pdf  
  *Historical EUA prices reaching 60–100+ €/tCO₂.*

- **EEX – EU ETS EUA Spot Market**  
  https://www.eex.com/en/market-data/environmental-markets/spot-market  
  *Confirms real EUA price volatility around 60–100 €/tCO₂.*

---

### **5. Short-haul vs long-haul / flight-phase inefficiency**

These support your market segment & flight-phase insights.

- **EUROCONTROL – Emissions by Flight Phase**  
  https://ansperformance.eu/environment/emissions/  
  *Cruise dominates total CO₂, but climb/descent disproportionately affect short-haul.*

- **ICCT – Global Aviation CO₂ by Flight Distance**  
  https://theicct.org/publication/global-aviation-ptf-co2-2022/  
  *Short-haul flights have higher CO₂/km due to phase distribution.*

---

### **6. Market segment behaviour (WTP & corporate SAF)**

These support your “B2B-heavy” vs “price-sensitive” segments.

- **IATA – Corporate SAF / Book-and-Claim Programs**  
  https://www.iata.org/en/programs/environment/saf/#tab-4  
  *Corporate buyers (Microsoft, DHL, Amazon, etc.) drive early SAF adoption.*

- **ICAO – Long-Term Aspirational Goal (LTAG)**  
  https://www.icao.int/environmental-protection/pages/icao_ltaga.aspx  
  *Highlights the role of policy & market incentives in SAF adoption across segments.*


## 0. Setup

We first import core Python libraries:

- `pandas` and `numpy` for data manipulation
- `matplotlib` for plots
- `pathlib` for robust file paths

We also define a base directory pointing to the Datathon `01-raw_data` folder, which hosts:

- Eurostat fuel/energy data
- EUROCONTROL CO₂ by state
- EUROCONTROL gate-to-gate (G2G) emissions


In [2]:
# ============================================
# STEP 0 — Imports & Global Config
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

base_dir = Path().resolve().parent
raw_data_path = base_dir / "data" / "01-raw_data"


In [3]:
# ============================================
# STEP 1 — Load Eurostat aviation energy & build fuel demand (Mt)
# ============================================

energy_path = raw_data_path / "eu-27_final energy consumption for different fuels used in air transport.csv"

df_energy = pd.read_csv(
    energy_path,
    skiprows=7,
    header=0,
    sep=",",
    engine="python"
)

# Keep only EU-27 series columns
eu27_cols = [col for col in df_energy.columns if "EU" in col]

df_energy = (
    df_energy[eu27_cols]
    .drop("Timeseries_EU-27_x1", axis=1)
    .apply(pd.to_numeric, errors="coerce")
)

# Index as years: 1990–2022
df_energy.index = range(1990, 2023)
df_energy.index.name = "Year"

df_energy.to_csv("eu27_historical_energy_consumption_from_1990_to_2022.csv")

display(df_energy.head())
display(df_energy.tail())
display(df_energy.info())


,Timeseries_EU-27_Biofuels_(Domestic_aviation),Timeseries_EU-27_Biofuels_(International_aviation),Timeseries_EU-27_Kerosene-type_jet_fuel_(Domestic_navigation),Timeseries_EU-27_Kerosene-type_jet_fuel_(International_navigation),Timeseries_EU-27_Other_fossil_fuels_(Domestic_aviation),Timeseries_EU-27_Other_fossil_fuels_(International_aviation)
Year,,,,,,
1990,0.0,0.0,170318.40,760870.036,38897.500,2016.620
1991,0.0,0.0,172418.23,751504.861,23699.689,570.032
1992,0.0,0.0,175020.60,806155.469,20102.589,570.032
1993,0.0,0.0,172272.71,846298.746,15401.189,350.852
1994,0.0,0.0,167804.42,899820.118,8880.589,307.016


,Timeseries_EU-27_Biofuels_(Domestic_aviation),Timeseries_EU-27_Biofuels_(International_aviation),Timeseries_EU-27_Kerosene-type_jet_fuel_(Domestic_navigation),Timeseries_EU-27_Kerosene-type_jet_fuel_(International_navigation),Timeseries_EU-27_Other_fossil_fuels_(Domestic_aviation),Timeseries_EU-27_Other_fossil_fuels_(International_aviation)
Year,,,,,,
2018,0.000,0.184,259769.459,1717200.792,2492.834,12.634
2019,0.000,0.000,269850.930,1749732.852,2404.418,11.765
2020,0.000,0.000,124800.264,755269.407,2017.453,14.530
2021,0.000,0.000,170674.983,907280.013,2412.687,20.166
2022,325.924,1944.945,246955.403,1448381.856,2014.813,28.565


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 1990 to 2022
Data columns (total 6 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Timeseries_EU-27_Biofuels_(Domestic_aviation)                       33 non-null     float64
 1   Timeseries_EU-27_Biofuels_(International_aviation)                  33 non-null     float64
 2   Timeseries_EU-27_Kerosene-type_jet_fuel_(Domestic_navigation)       33 non-null     float64
 3   Timeseries_EU-27_Kerosene-type_jet_fuel_(International_navigation)  33 non-null     float64
 4   Timeseries_EU-27_Other_fossil_fuels_(Domestic_aviation)             33 non-null     float64
 5   Timeseries_EU-27_Other_fossil_fuels_(International_aviation)        33 non-null     float64
dtypes: float64(6)
memory usage: 1.7 KB


None

In [4]:
# Aggregate domestic + international, convert to Mt fuel
# Assumption: 0.043 TJ/kt → 1/(0.043*1e6) to go from TJ to Mt fuel
df_baseline = pd.DataFrame(
    {
        "Total_Domestic_Mt": df_energy[[c for c in df_energy if "Domestic" in c]].sum(axis=1),
        "Total_International_Mt": df_energy[[c for c in df_energy if "International" in c]].sum(axis=1),
        "Total_Fuel_Mt": df_energy.sum(axis=1)
    },
    index=df_energy.index
) * (1 / (0.043 * 1e6))

df_baseline.index.name = "Year"
df_fuel_hist = df_baseline.reset_index()[["Year", "Total_Fuel_Mt"]]

df_fuel_hist.to_csv("eu27_historical_aggregate_fuel_demand.csv", index=False)

display(df_fuel_hist.head())
display(df_fuel_hist.tail())


,Year,Total_Fuel_Mt
0,1990,22.607036
1,1991,22.050996
2,1992,23.298807
3,1993,24.054035
4,1994,25.042143


,Year,Total_Fuel_Mt
28,2018,46.034323
29,2019,47.023255
30,2020,20.513992
31,2021,25.125299
32,2022,39.526779


## 1. Historical EU27 fuel demand (Eurostat)

We use the Eurostat file:

> `eu-27_final energy consumption for different fuels used in air transport.csv`

Steps:

1. Filter columns corresponding to **EU-27** aggregate.
2. Convert all fields to numeric.
3. Index rows as **years 1990–2022**.
4. Aggregate domestic + international aviation and convert to **Mt of fuel** using:

\[
\text{Fuel (Mt)} = \frac{\text{Energy (TJ)}}{0.043 \times 10^6}
\]

Assumption:  
- Jet fuel energy content ≈ **0.043 PJ per kilotonne**, a standard value used in IPCC/EEA fuel statistics.

This gives us a **historical fuel baseline** (`df_fuel_hist`) for EU27 aviation.

**Source (conceptual):**  
- Eurostat – Final energy consumption in air transport (EU27)  
- Typical jet fuel energy content from IPCC/EEA methodologies.


In [5]:
# ============================================
# STEP 2 — Project fuel demand to 2050 (constant growth)
# ============================================

GROWTH_RATE = 0.016        # +1.6% AAGR (Eurocontrol base)
START_YEAR = 2024
END_YEAR = 2050
BASELINE_FUEL_MT = 39.0    # ~2024/2025 baseline

years = np.arange(START_YEAR, END_YEAR + 1)
df_projection = pd.DataFrame({"Year": years}).set_index("Year")

# initialize
df_projection.loc[START_YEAR, "Total_Fuel_Mt"] = BASELINE_FUEL_MT

# CAGR projection
for year in years[1:]:
    prev = df_projection.loc[year - 1, "Total_Fuel_Mt"]
    df_projection.loc[year, "Total_Fuel_Mt"] = prev * (1 + GROWTH_RATE)

print("--- Projected Total Fuel Demand (2024–2050) ---")
display(df_projection.head())
display(df_projection.tail())


--- Projected Total Fuel Demand (2024–2050) ---


,Total_Fuel_Mt
Year,
2024,39.000000
2025,39.624000
2026,40.257984
2027,40.902112
2028,41.556546


,Total_Fuel_Mt
Year,
2046,55.300134
2047,56.184936
2048,57.083895
2049,57.997237
2050,58.925193


In [6]:
# Combine historical + 2023 estimate + projections
df_fuel_hist_2023 = df_fuel_hist.copy()
df_fuel_hist_2023 = pd.concat(
    [
        df_fuel_hist_2023,
        pd.DataFrame({"Year": [2023], "Total_Fuel_Mt": [38.7]})
    ],
    ignore_index=True
)

df_fuel_proj = df_projection.reset_index()

df_fuel_full = (
    pd.concat([df_fuel_hist_2023, df_fuel_proj], ignore_index=True)
    .sort_values("Year")
    .reset_index(drop=True)
)

print("Unified fuel series 1990–2050:")
display(df_fuel_full.head())
display(df_fuel_full.tail())


Unified fuel series 1990–2050:


,Year,Total_Fuel_Mt
0,1990,22.607036
1,1991,22.050996
2,1992,23.298807
3,1993,24.054035
4,1994,25.042143


,Year,Total_Fuel_Mt
56,2046,55.300134
57,2047,56.184936
58,2048,57.083895
59,2049,57.997237
60,2050,58.925193


## 2. Fuel demand projections (2024–2050)

To project aviation fuel demand, we adopt a **simple but defensible approach**:

- Constant **annual growth rate (AAGR)** = **1.6%**
- Baseline fuel in mid-2020s ≈ **39 Mt**
- Years: **2024–2050**

This is consistent with **EUROCONTROL’s base traffic scenario**, which suggests moderate long-term growth in EU air traffic after COVID recovery.

We also insert a single estimate for **2023 (38.7 Mt)** to bridge historical and projected data.

The result is a **unified fuel series 1990–2050** (`df_fuel_full`) which becomes the core input for the SAF scenarios.

**Why 1.6%?**

- EUROCONTROL’s long-term outlook indicates that EU flights and CO₂ grow in the range of ~1.5–2.0% after stabilisation, which is consistent with historical aviation–GDP elasticities.


In [7]:
# ============================================
# STEP 3 — CO₂ by state (EU27 aggregate)
# ============================================

eu27_names = [
    "AUSTRIA", "BELGIUM", "BULGARIA", "CROATIA", "CYPRUS", "CZECHIA",
    "DENMARK*", "ESTONIA", "FINLAND", "FRANCE", "GERMANY", "GREECE",
    "HUNGARY", "IRELAND", "ITALY", "LATVIA", "LITHUANIA", "LUXEMBOURG",
    "MALTA", "NETHERLANDS", "POLAND", "PORTUGAL*", "ROMANIA",
    "SLOVAKIA", "SLOVENIA", "SPAIN", "SWEDEN"
]

co2_path = raw_data_path / "CO2_emissions_by_state.xlsx"
df_co2_state = pd.read_excel(co2_path, sheet_name="DATA")

df_eu27 = df_co2_state[df_co2_state["STATE_NAME"].isin(eu27_names)]

df_co2_emission = (
    df_eu27
    .groupby("YEAR")[["CO2_QTY_TONNES", "TF"]]
    .sum()
    .reset_index()
    .rename(columns={"YEAR": "Year"})
)

df_co2_emission["CO2_Mt_state"] = df_co2_emission["CO2_QTY_TONNES"] / 1e6
df_co2_ts = df_co2_emission[["Year", "CO2_Mt_state", "TF"]]

print("EU27 CO₂ by year:")
display(df_co2_ts.head())
display(df_co2_ts.tail())


EU27 CO₂ by year:


,Year,CO2_Mt_state,TF
0,2010,112.891825,6033208
1,2011,117.617216,6213612
2,2012,112.793098,5946172
3,2013,111.703111,5750751
4,2014,115.512675,5836750


,Year,CO2_Mt_state,TF
11,2021,75.652978,3796930
12,2022,114.586806,5677316
13,2023,128.558424,6112174
14,2024,138.984989,6417359
15,2025,111.019203,5084778


## 3. CO₂ emissions by state (EU27 aggregate)

We now integrate the EUROCONTROL **CO₂ by state** dataset:

> `CO2_emissions_by_state.xlsx` (sheet: `DATA`)

Steps:

1. Filter **EU27 countries** (list of Member States).
2. Group by `YEAR` and sum:
   - `CO2_QTY_TONNES` → total aviation CO₂
   - `TF` → total flights
3. Convert CO₂ to **Mt** (`CO2_Mt_state`).

This dataset provides an **independent CO₂ view** to compare against:

- Our fuel-based calculation  
- G2G emissions by flight phase/segment  

This strengthens the **calibration and plausibility checks** of the model.


In [8]:
# ============================================
# STEP 4 — G2G emissions (EU27, departing flights only)
# ============================================

g2g_path = raw_data_path / "g2g_emissions.xlsx"
df_g2g = pd.read_excel(g2g_path, sheet_name="DATA")

# EU27, domestic (departing) flights
df_g2g = df_g2g[(df_g2g["AREA"] == "EU27") & (df_g2g["FLIGHT_TYPE"] == "D")]
df_g2g = df_g2g.drop(["LEVEL", "AREA", "FOCUS_TYPE", "FLIGHT_TYPE"], axis=1).reset_index(drop=True)

display(df_g2g.head())


,YEAR,MONTH,MARKET_SEGMENT,FLIGHT_PHASE,NB_FLIGHTS,CO2_TONS,NOX_KG,SOX_KG
0,2019,1,business,cruise,7137,23805.1,76778.89,6329.94
1,2019,1,business,descent,7137,85.0,138.62,22.51
2,2019,1,business,climb,7137,2409.7,9573.80,637.87
3,2019,1,business,taxi-in,7137,0.3,0.31,0.07
4,2019,1,business,taxi-out,7137,1252.0,1129.37,330.04


In [9]:
# 4.1 Year × segment × phase aggregates
df_seg_phase = (
    df_g2g
    .groupby(["YEAR", "MARKET_SEGMENT", "FLIGHT_PHASE"])
    .agg({
        "NB_FLIGHTS": "sum",
        "CO2_TONS": "sum",
        "NOX_KG": "sum",
        "SOX_KG": "sum"
    })
    .reset_index()
)

display(df_seg_phase.head())


,YEAR,MARKET_SEGMENT,FLIGHT_PHASE,NB_FLIGHTS,CO2_TONS,NOX_KG,SOX_KG
0,2019,business,climb,109903,37933.7,150725.04,10033.94
1,2019,business,cruise,109903,386847.4,1257313.99,102885.71
2,2019,business,descent,109903,1194.3,1711.97,316.42
3,2019,business,taxi-in,92988,1.5,1.65,0.43
4,2019,business,taxi-out,109903,20623.5,18960.51,5439.81


In [10]:
# 4.2 Yearly totals (CO2, flights, NOx, SOx)
df_g2g_year = (
    df_seg_phase
    .groupby("YEAR")[["CO2_TONS", "NB_FLIGHTS", "NOX_KG", "SOX_KG"]]
    .sum()
    .reset_index()
    .rename(columns={
        "YEAR": "Year",
        "CO2_TONS": "CO2_TONS_g2g",
        "NB_FLIGHTS": "NB_FLIGHTS_g2g",
        "NOX_KG": "NOX_KG_g2g",
        "SOX_KG": "SOX_KG_g2g"
    })
)

df_g2g_year["CO2_per_flight_kg_avg"] = (
    df_g2g_year["CO2_TONS_g2g"] * 1000 / df_g2g_year["NB_FLIGHTS_g2g"]
)

print("Yearly G2G aggregates:")
display(df_g2g_year.head())


Yearly G2G aggregates:


,Year,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg
0,2019,35257019.7,9058821,2.021278e+08,9400112.51,3892.009755
1,2020,14766529.1,3288494,8.810524e+07,3936686.41,4490.362184
2,2021,17569693.3,3732895,1.034831e+08,4683967.55,4706.720468
3,2022,28385620.8,6813508,1.620048e+08,7567687.32,4166.080204
4,2023,32595198.0,7680746,1.869263e+08,8690070.31,4243.754187


In [11]:
# 4.3 Phase totals & shares
df_phase_totals = (
    df_seg_phase
    .groupby(["YEAR", "FLIGHT_PHASE"])["CO2_TONS"]
    .sum()
    .reset_index()
    .rename(columns={"YEAR": "Year"})
)

df_phase_totals["CO2_SHARE"] = (
    df_phase_totals
    .groupby("Year")["CO2_TONS"]
    .transform(lambda x: x / x.sum())
)

display(df_phase_totals.head())


,Year,FLIGHT_PHASE,CO2_TONS,CO2_SHARE
0,2019,climb,2281810.1,0.064719
1,2019,cruise,31674240.2,0.898381
2,2019,descent,9349.2,0.000265
3,2019,taxi-in,130.0,0.000004
4,2019,taxi-out,1291490.2,0.036631


In [12]:
# 4.4 CO₂ per flight by segment & phase
df_seg_phase["CO2_per_flight_kg"] = (
    df_seg_phase["CO2_TONS"] * 1000 / df_seg_phase["NB_FLIGHTS"]
)

display(
    df_seg_phase[["YEAR", "MARKET_SEGMENT", "FLIGHT_PHASE", "CO2_per_flight_kg"]]
    .head()
)


,YEAR,MARKET_SEGMENT,FLIGHT_PHASE,CO2_per_flight_kg
0,2019,business,climb,345.156183
1,2019,business,cruise,3519.898456
2,2019,business,descent,10.866855
3,2019,business,taxi-in,0.016131
4,2019,business,taxi-out,187.651838


In [13]:
# 4.5 Pollutants (NOx, SOx)
df_pollutants = (
    df_seg_phase
    .groupby("YEAR")[["NOX_KG", "SOX_KG"]]
    .sum()
    .reset_index()
    .rename(columns={"YEAR": "Year"})
)

df_pollutants["NOX_tonnes"] = df_pollutants["NOX_KG"] / 1000
df_pollutants["SOX_tonnes"] = df_pollutants["SOX_KG"] / 1000

display(df_pollutants.head())


,Year,NOX_KG,SOX_KG,NOX_tonnes,SOX_tonnes
0,2019,2.021278e+08,9400112.51,202127.81034,9400.11251
1,2020,8.810524e+07,3936686.41,88105.24477,3936.68641
2,2021,1.034831e+08,4683967.55,103483.05959,4683.96755
3,2022,1.620048e+08,7567687.32,162004.82972,7567.68732
4,2023,1.869263e+08,8690070.31,186926.32968,8690.07031


## 4. Gate-to-Gate (G2G) emissions: segments, phases, pollutants

We use EUROCONTROL’s **G2G emissions dataset**:

> `g2g_emissions.xlsx` (sheet: `DATA`)

Filtered to:

- `AREA = "EU27"`  
- `FLIGHT_TYPE = "D"` (departing flights, matching Datathon focus)

We compute:

1. **Year × Market Segment × Flight Phase** (`df_seg_phase`):  
   - `NB_FLIGHTS`  
   - `CO2_TONS`  
   - `NOX_KG`, `SOX_KG`
2. **Yearly totals** (`df_g2g_year`):  
   - Total CO₂, flights, NOx, SOx  
   - Average **CO₂ per flight (kg)**.
3. **Phase-level totals & shares** (`df_phase_totals`):  
   - For each year, the share of CO₂ by phase (climb, cruise, descent, taxi-in/out).
4. **Per-flight CO₂ by segment & phase**:  
   - Useful to identify where operational or technological measures matter most.
5. **Pollutant totals** (`df_pollutants`):  
   - NOx and SOx in tonnes per year.

Although these variables are **not required** in the final submission dataset, they:

- Enrich our **calibration table**
- Provide insight into **non-CO₂ climate impacts**
- Support additional narratives and plots for Objective 2 and 3 (e.g. short-haul vs long-haul, flight phase focus).

**Conceptual source:**  
EUROCONTROL ANS Performance – Environment / Emissions dashboards.


In [14]:
# ============================================
# STEP 5 — Final calibration table (fuel + CO2 + segments + phases)
# ============================================

# Start from unified fuel series
calibration = df_fuel_full.copy()   # Year, Total_Fuel_Mt

# Add CO2 by state
calibration = calibration.merge(df_co2_ts, on="Year", how="left")

# Add G2G yearly aggregates
calibration = calibration.merge(df_g2g_year, on="Year", how="left")

# Theoretical CO2 from fuel
EF_JET = 3.15  # tCO2 / tonne fuel
calibration["CO2_Mt_calc"] = calibration["Total_Fuel_Mt"] * EF_JET

calibration = calibration.sort_values("Year").reset_index(drop=True)
calibration.to_csv("eu27_calibration_table_1990_2050.csv", index=False)
print("=== CALIBRATION (fuel + state CO2 + g2g) ===")
display(calibration.head(15))
display(calibration.tail(15))


=== CALIBRATION (fuel + state CO2 + g2g) ===


,Year,Total_Fuel_Mt,CO2_Mt_state,TF,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg,CO2_Mt_calc
0,1990,22.607036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.212164
1,1991,22.050996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.460636
2,1992,23.298807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.391241
3,1993,24.054035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.770210
4,1994,25.042143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.882750
5,1995,26.105702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.232960
6,1996,27.175963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.604282
7,1997,28.505274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.791614
8,1998,29.989791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.467841
9,1999,32.037534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.918232


,Year,Total_Fuel_Mt,CO2_Mt_state,TF,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg,CO2_Mt_calc
46,2036,47.183386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.627665
47,2037,47.938320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.005708
48,2038,48.705333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.421799
49,2039,49.484618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.876548
50,2040,50.276372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158.370573
51,2041,51.080794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.904502
52,2042,51.898087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163.478974
53,2043,52.728456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.094638
54,2044,53.572112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.752152
55,2045,54.429266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171.452186


In [15]:
# 5.1 Add activity & segment/phase breakdown

# Year × segment
df_segment_year = (
    df_g2g
    .groupby(["YEAR", "MARKET_SEGMENT"])
    .agg({"NB_FLIGHTS": "sum", "CO2_TONS": "sum"})
    .reset_index()
    .rename(columns={"YEAR": "Year"})
)

# Year × phase
df_phase_year = (
    df_g2g
    .groupby(["YEAR", "FLIGHT_PHASE"])["CO2_TONS"]
    .sum()
    .reset_index()
    .rename(columns={"YEAR": "Year"})
)

# Activity (flights per year)
df_activity_year = (
    df_g2g
    .groupby("YEAR")["NB_FLIGHTS"]
    .sum()
    .reset_index()
    .rename(columns={"YEAR": "Year", "NB_FLIGHTS": "NB_FLIGHTS_g2g_full"})
)

# Merge activity
calibration = calibration.merge(df_activity_year, on="Year", how="left")

# Pivot segments -> columns
df_segment_pivot = (
    df_segment_year
    .pivot(index="Year", columns="MARKET_SEGMENT", values="CO2_TONS")
    .fillna(0)
)
df_segment_pivot.columns = [f"CO2_seg_{c}" for c in df_segment_pivot.columns]

calibration = calibration.merge(df_segment_pivot, on="Year", how="left")

# Pivot phases -> columns
df_phase_pivot = (
    df_phase_year
    .pivot(index="Year", columns="FLIGHT_PHASE", values="CO2_TONS")
    .fillna(0)
)
df_phase_pivot.columns = [f"CO2_phase_{c}" for c in df_phase_pivot.columns]

calibration = calibration.merge(df_phase_pivot, on="Year", how="left")

calibration.to_csv("eu27_calibration_table_1990_2050.csv", index=False)

print("=== FINAL CALIBRATION TABLE (1990–2050) ===")
display(calibration.head(20))
display(calibration.tail(20))


=== FINAL CALIBRATION TABLE (1990–2050) ===


,Year,Total_Fuel_Mt,CO2_Mt_state,TF,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg,CO2_Mt_calc,...,CO2_seg_cargo,CO2_seg_lowcost,CO2_seg_mainline,CO2_seg_other,CO2_seg_regional,CO2_phase_climb,CO2_phase_cruise,CO2_phase_descent,CO2_phase_taxi-in,CO2_phase_taxi-out
0,1990,22.607036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.212164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1991,22.050996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.460636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992,23.298807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.391241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993,24.054035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.770210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1994,25.042143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.882750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1995,26.105702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.232960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1996,27.175963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.604282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,28.505274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.791614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1998,29.989791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.467841,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1999,32.037534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.918232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Year,Total_Fuel_Mt,CO2_Mt_state,TF,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg,CO2_Mt_calc,...,CO2_seg_cargo,CO2_seg_lowcost,CO2_seg_mainline,CO2_seg_other,CO2_seg_regional,CO2_phase_climb,CO2_phase_cruise,CO2_phase_descent,CO2_phase_taxi-in,CO2_phase_taxi-out
41,2031,43.583345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.287538,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,2032,44.280679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.484138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,2033,44.989170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.715885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2034,45.708996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143.983339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,2035,46.440340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146.287072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,2036,47.183386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.627665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2037,47.938320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.005708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2038,48.705333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.421799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2039,49.484618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.876548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2040,50.276372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158.370573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5. Calibration table (1990–2050)

We now build a single **calibration table** by merging:

1. `df_fuel_full` → **fuel demand** (Mt, 1990–2050)
2. `df_co2_ts` → **CO₂ by state** (Mt, TF)
3. `df_g2g_year` → **CO₂, flights, NOx, SOx** from G2G
4. `df_activity_year` → total flights from G2G
5. `df_segment_pivot` → CO₂ by **market segment** (BUSINESS, LOWCOST, CARGO, etc.)
6. `df_phase_pivot` → CO₂ by **flight phase** (CLIMB, CRUISE, etc.)
7. `CO2_Mt_calc` → theoretical CO₂ = `Total_Fuel_Mt × 3.15 tCO₂/t`

The resulting `calibration` DataFrame contains, for each year:

- Fuel demand (Mt)
- Emissions (state-reported, G2G, theoretical)
- Flights and pollutants
- Market structure (segment)
- Operational structure (flight phase)

This acts as the **master dataset** for checking consistency and interpreting the SAF scenarios (e.g. which segments/phases would benefit most from SAF or operational measures).

**Emission factor assumption:**

- EF_JET = **3.15 tCO₂ per tonne jet fuel**, widely used in ICAO / IPCC aviation calculations.


In [16]:
# ============================================
# STEP 6 — SAF scenarios: S0 (BAU) & S1 (Policy)
# ============================================

EF_JET = 3.15
SAF_LCA_REDUCTION = 0.75
EF_SAF = EF_JET * (1 - SAF_LCA_REDUCTION)

START_YEAR_MODEL = 2025
END_YEAR_MODEL = 2050

def saf_share_s0(year: int) -> float:
    """BAU: ~2% in 2025 to ~30% in 2050"""
    if year < 2025:
        return 0.01
    if year > 2050:
        year = 2050
    return 0.02 + (0.30 - 0.02) * (year - 2025) / (2050 - 2025)

refeuleu_floors = {2025: 0.02, 2030: 0.06, 2035: 0.20, 2050: 0.70}

def saf_floor_s1(year: int) -> float:
    years = sorted(refeuleu_floors.keys())
    if year <= years[0]:
        return refeuleu_floors[years[0]]
    if year >= years[-1]:
        return refeuleu_floors[years[-1]]
    for y0, y1 in zip(years[:-1], years[1:]):
        if y0 <= year <= y1:
            v0, v1 = refeuleu_floors[y0], refeuleu_floors[y1]
            return v0 + (v1 - v0) * (year - y0) / (y1 - y0)

def saf_share_s1(year: int) -> float:
    """Policy scenario: at least ReFuelEU, +10pp by 2050."""
    floor = saf_floor_s1(year)
    extra = 0.10 * (year - 2025) / (2050 - 2025)
    return min(1.0, floor + max(0, extra))


In [17]:
rows = []

for year in range(START_YEAR_MODEL, END_YEAR_MODEL + 1):
    row_cal = calibration[calibration["Year"] == year]
    if row_cal.empty:
        continue

    total_fuel_mt = float(row_cal["Total_Fuel_Mt"].iloc[0])
    co2_baseline_mt = total_fuel_mt * EF_JET   # 0% SAF baseline

    for scenario in [0, 1]:
        if scenario == 0:
            saf_share = saf_share_s0(year)
        else:
            saf_share = saf_share_s1(year)

        saf_share = max(0.0, min(1.0, saf_share))
        jet_frac = 1 - saf_share
        saf_frac = saf_share

        co2_emissions_mt = total_fuel_mt * (
            jet_frac * EF_JET + saf_frac * EF_SAF
        )
        avoided_mt = co2_baseline_mt - co2_emissions_mt

        rows.append({
            "Year": year,
            "Scenario": scenario,
            "Total_Fuel_Mt": total_fuel_mt,
            "SAF_Share_frac": saf_share,
            "CO2_Emissions_Mt": co2_emissions_mt,
            "CO2_Baseline_no_SAF_Mt": co2_baseline_mt,
            "Avoided_CO2_Mt": avoided_mt,
        })

df_saf_model = pd.DataFrame(rows).sort_values(["Year", "Scenario"])

display(df_saf_model.head())
display(df_saf_model.tail())


,Year,Scenario,Total_Fuel_Mt,SAF_Share_frac,CO2_Emissions_Mt,CO2_Baseline_no_SAF_Mt,Avoided_CO2_Mt
0,2025,0,39.624000,0.0200,122.943366,124.815600,1.872234
1,2025,1,39.624000,0.0200,122.943366,124.815600,1.872234
2,2026,0,40.257984,0.0312,123.845234,126.812650,2.967416
3,2026,1,40.257984,0.0320,123.769146,126.812650,3.043504
4,2027,0,40.902112,0.0424,124.744487,128.841652,4.097165


,Year,Scenario,Total_Fuel_Mt,SAF_Share_frac,CO2_Emissions_Mt,CO2_Baseline_no_SAF_Mt,Avoided_CO2_Mt
47,2048,1,57.083895,0.725333,81.995307,179.814269,97.818962
48,2049,0,57.997237,0.288800,143.120362,182.691297,39.570935
49,2049,1,57.997237,0.762667,78.191875,182.691297,104.499422
50,2050,0,58.925193,0.300000,143.851127,185.614358,41.763231
51,2050,1,58.925193,0.800000,74.245743,185.614358,111.368615


## 6. SAF scenarios — S0 (BAU) & S1 (Policy / ReFuelEU)

We define two stylised SAF adoption paths:

### Scenario 0 — S0 (Business-as-Usual)

- SAF share is low and grows slowly.
- Approximately:
  - ~2% SAF share in 2025
  - ~30% by 2050
- Represents a world where SAF deployment is constrained by cost, infrastructure, or weak policy.

### Scenario 1 — S1 (Policy / ReFuelEU-aligned)

- Based on **ReFuelEU Aviation** minimum SAF blending mandates, approximated as:
  - 2% in 2025  
  - 6% in 2030  
  - 20% in 2035  
  - 70% in 2050
- We add a smooth +10 percentage points by 2050 to reflect **additional ambition** beyond the minimum floors.

### Emissions methodology

For each year and scenario:

- Use `Total_Fuel_Mt` from the calibration table.
- Let:
  - EF_JET = **3.15 tCO₂/t** (fossil jet)  
  - SAF lifecycle reduction = **75%** (midpoint of 70–80% band suggested in the Datathon guide).
- Then:
  - EF_SAF = EF_JET × (1 − 0.75)  
  - CO₂_baseline (0% SAF) = `Total_Fuel_Mt × EF_JET`
  - CO₂_scenario = `Total_Fuel_Mt × [ (1−SAF_share) × EF_JET + SAF_share × EF_SAF ]`
  - Avoided CO₂ = baseline − scenario

**Justification for 75% LCA reduction:**

- Commonly cited in:
  - IATA SAF factsheets  
  - ICAO CORSIA LCA documentation  
  - EASA / ICCT assessments  
- The guide explicitly suggested using 70–80% and stating our choice; we take the midpoint (75%) for transparency.


In [18]:
# ============================================
# STEP 7 — Datathon output: Year, Scenario, Total_Fuel, SAF_Share, CO2_Emissions, Avoided_CO2
# ============================================

df_output_final = df_saf_model.copy()
df_output_final["SAF_Share"] = df_output_final["SAF_Share_frac"] * 100

df_output_final = df_output_final[[
    "Year",
    "Scenario",
    "Total_Fuel_Mt",
    "SAF_Share",
    "CO2_Emissions_Mt",
    "Avoided_CO2_Mt"
]].rename(columns={
    "Total_Fuel_Mt": "Total_Fuel",
    "CO2_Emissions_Mt": "CO2_Emissions",
    "Avoided_CO2_Mt": "Avoided_CO2"
})

display(df_output_final.head())
display(df_output_final.tail())

df_output_final.to_csv("ie_iata_output_dataset.csv", index=False)


,Year,Scenario,Total_Fuel,SAF_Share,CO2_Emissions,Avoided_CO2
0,2025,0,39.624000,2.00,122.943366,1.872234
1,2025,1,39.624000,2.00,122.943366,1.872234
2,2026,0,40.257984,3.12,123.845234,2.967416
3,2026,1,40.257984,3.20,123.769146,3.043504
4,2027,0,40.902112,4.24,124.744487,4.097165


,Year,Scenario,Total_Fuel,SAF_Share,CO2_Emissions,Avoided_CO2
47,2048,1,57.083895,72.533333,81.995307,97.818962
48,2049,0,57.997237,28.880000,143.120362,39.570935
49,2049,1,57.997237,76.266667,78.191875,104.499422
50,2050,0,58.925193,30.000000,143.851127,41.763231
51,2050,1,58.925193,80.000000,74.245743,111.368615


## 7. Datathon output dataset

The Datathon requires a **clean output table** with these columns:

- `Year`  
- `Scenario`  
- `Total_Fuel` (Mt)  
- `SAF_Share` (%)  
- `CO2_Emissions` (Mt)  
- `Avoided_CO2` (Mt vs 0% SAF baseline)

In this step, we:

1. Convert `SAF_Share_frac` → `SAF_Share` (%).
2. Rename columns to match the required schema.
3. Export:

> `ie_iata_output_dataset.csv`

This is **the main dataset** the jury will use to validate our results and generate any additional checks.


In [19]:
# ============================================
# STEP 8 — Probability-weighted expected outcomes
# ============================================

p_s0 = 0.7  # BAU
p_s1 = 0.3  # Policy
prob_map = {0: p_s0, 1: p_s1}

df_prob = df_saf_model.copy()
df_prob["Scenario_Prob"] = df_prob["Scenario"].map(prob_map)

rows = []
for year, grp in df_prob.groupby("Year"):
    total_fuel = grp["Total_Fuel_Mt"].iloc[0]
    exp_saf_share = (grp["SAF_Share_frac"] * grp["Scenario_Prob"]).sum()
    exp_emissions = (grp["CO2_Emissions_Mt"] * grp["Scenario_Prob"]).sum()
    exp_avoided = (grp["Avoided_CO2_Mt"] * grp["Scenario_Prob"]).sum()

    rows.append({
        "Year": year,
        "Expected_Total_Fuel_Mt": total_fuel,
        "Expected_SAF_Share_frac": exp_saf_share,
        "Expected_CO2_Emissions_Mt": exp_emissions,
        "Expected_Avoided_CO2_Mt": exp_avoided
    })

df_prob_weighted = pd.DataFrame(rows).sort_values("Year")
df_prob_weighted["Expected_SAF_Share_pct"] = df_prob_weighted["Expected_SAF_Share_frac"] * 100

print("=== Probability-weighted expected outcome ===")
display(df_prob_weighted.head(10))
display(df_prob_weighted.tail(10))

df_prob_weighted.to_csv("ie_iata_prob_weighted_results.csv", index=False)


=== Probability-weighted expected outcome ===


,Year,Expected_Total_Fuel_Mt,Expected_SAF_Share_frac,Expected_CO2_Emissions_Mt,Expected_Avoided_CO2_Mt,Expected_SAF_Share_pct
0,2025,39.624000,0.02000,122.943366,1.872234,2.000
1,2026,40.257984,0.03144,123.822407,2.990242,3.144
2,2027,40.902112,0.04288,124.698104,4.143548,4.288
3,2028,41.556546,0.05432,125.570125,5.332993,5.432
4,2029,42.221450,0.06576,126.438128,6.559440,6.576
5,2030,42.896993,0.07720,127.301761,7.823768,7.720
6,2031,43.583345,0.09464,127.542868,9.744669,9.464
7,2032,44.280679,0.11208,127.759102,11.725037,11.208
8,2033,44.989170,0.12952,127.949604,13.766281,12.952
9,2034,45.708996,0.14696,128.113495,15.869844,14.696


,Year,Expected_Total_Fuel_Mt,Expected_SAF_Share_frac,Expected_CO2_Emissions_Mt,Expected_Avoided_CO2_Mt,Expected_SAF_Share_pct
16,2041,51.080794,0.27864,127.278679,33.625823,27.864
17,2042,51.898087,0.29768,126.980658,36.498316,29.768
18,2043,52.728456,0.31672,126.640518,39.454120,31.672
19,2044,53.572112,0.33576,126.256985,42.495167,33.576
20,2045,54.429266,0.35480,125.828760,45.623427,35.480
21,2046,55.300134,0.37384,125.354509,48.840912,37.384
22,2047,56.184936,0.39288,124.832870,52.149678,39.288
23,2048,57.083895,0.41192,124.262449,55.551820,41.192
24,2049,57.997237,0.43096,123.641816,59.049481,43.096
25,2050,58.925193,0.45000,122.969512,62.644846,45.000


## 8. Probability-weighted outcomes (IEA-inspired)

The IEA recently warned that **fossil fuel demand may continue growing for ~25 years** without major new policies, implying that **higher-warming / BAU pathways are currently more likely** than low-warming ones.

To reflect this in a simple way:

- We assign **70% probability** to **Scenario 0 (BAU)**  
- And **30% probability** to **Scenario 1 (Policy)**

For each year we compute:

- Expected total fuel (unchanged by scenario in this setup)
- Expected SAF share (probability-weighted average)
- Expected CO₂ emissions
- Expected avoided CO₂

This gives a **realistic expectation path**, not just two extreme worlds.

We export this as:

> `ie_iata_prob_weighted_results.csv`

**Rationale:**

- Directly aligned with the mentor’s email and the IEA warning (FT article referenced there).
- Transparent, easy to explain, and clearly labelled as an assumption.


In [20]:
# ============================================
# STEP 9 — Cost model & sensitivity
# ============================================

def run_cost_sensitivity(
    df_saf,
    P_jet_base=800,     # €/t jet fuel
    spread=900,         # SAF premium €/t
    ETS_price=100,      # €/tCO2
    SAF_LCA=0.75,       # fraction reduction
    EF_JET=3.15         # tCO2 / t fuel
):
    df = df_saf.copy()

    EF_SAF = EF_JET * (1 - SAF_LCA)

    P_SAF_base = P_jet_base + spread
    P_jet_eff = P_jet_base + EF_JET * ETS_price
    P_SAF_eff = P_SAF_base   # assume SAF exempt from ETS

    df["Fuel_jet_Mt"] = df["Total_Fuel_Mt"] * (1 - df["SAF_Share_frac"])
    df["Fuel_SAF_Mt"] = df["Total_Fuel_Mt"] * df["SAF_Share_frac"]

    df["Cost_baseline"] = df["Total_Fuel_Mt"] * 1e6 * P_jet_eff

    df["Cost_with_SAF"] = (
        df["Fuel_jet_Mt"] * 1e6 * P_jet_eff +
        df["Fuel_SAF_Mt"] * 1e6 * P_SAF_eff
    )

    df["Extra_cost"] = df["Cost_with_SAF"] - df["Cost_baseline"]

    df["CO2_Emissions_Mt_LCA"] = df["Total_Fuel_Mt"] * (
        (1 - df["SAF_Share_frac"]) * EF_JET + df["SAF_Share_frac"] * EF_SAF
    )
    df["Avoided_CO2_Mt_LCA"] = df["CO2_Baseline_no_SAF_Mt"] - df["CO2_Emissions_Mt_LCA"]

    df["Cost_per_tCO2"] = np.where(
        df["Avoided_CO2_Mt_LCA"] > 0,
        df["Extra_cost"] / (df["Avoided_CO2_Mt_LCA"] * 1e6),
        np.nan
    )

    df["P_jet_eff"] = P_jet_eff
    df["P_SAF_eff"] = P_SAF_eff
    df["SAF_LCA"] = SAF_LCA
    df["ETS_price"] = ETS_price
    df["spread"] = spread

    return df


In [21]:
spreads = [600, 900, 1200]
ets_prices = [50, 100, 150]
saf_lcas = [0.6, 0.75, 0.9]

sensitivity_rows = []

df_s1 = df_saf_model[df_saf_model["Scenario"] == 1].copy()

for spread in spreads:
    for ets in ets_prices:
        for lca in saf_lcas:
            df_sens = run_cost_sensitivity(
                df_s1,
                P_jet_base=800,
                spread=spread,
                ETS_price=ets,
                SAF_LCA=lca
            )
            for target_year in [2030, 2040, 2050]:
                df_y = df_sens[df_sens["Year"] == target_year]
                if df_y.empty:
                    continue
                row = df_y.iloc[0]

                sensitivity_rows.append({
                    "Year": target_year,
                    "spread_€/t": spread,
                    "ETS_price_€/tCO2": ets,
                    "SAF_LCA_reduction": lca,
                    "SAF_Share_%": row["SAF_Share_frac"] * 100,
                    "Extra_cost_€": row["Extra_cost"],
                    "Avoided_CO2_Mt_LCA": row["Avoided_CO2_Mt_LCA"],
                    "Cost_per_tCO2_€/t": row["Cost_per_tCO2"]
                })

df_sensitivity = (
    pd.DataFrame(sensitivity_rows)
    .sort_values(["Year", "spread_€/t", "ETS_price_€/tCO2", "SAF_LCA_reduction"])
    .reset_index(drop=True)
)

display(df_sensitivity.head(20))
display(df_sensitivity.tail(20))

df_sensitivity.to_csv("ie_iata_sensitivity_results.csv", index=False)


,Year,spread_€/t,ETS_price_€/tCO2,SAF_LCA_reduction,SAF_Share_%,Extra_cost_€,Avoided_CO2_Mt_LCA,Cost_per_tCO2_€/t
0,2030,600,50,0.60,8.0,1.518554e+09,6.486025,234.126984
1,2030,600,50,0.75,8.0,1.518554e+09,8.107532,187.301587
2,2030,600,50,0.90,8.0,1.518554e+09,9.729038,156.084656
3,2030,600,100,0.60,8.0,9.780515e+08,6.486025,150.793651
4,2030,600,100,0.75,8.0,9.780515e+08,8.107532,120.634921
5,2030,600,100,0.90,8.0,9.780515e+08,9.729038,100.529101
6,2030,600,150,0.60,8.0,4.375493e+08,6.486025,67.460317
7,2030,600,150,0.75,8.0,4.375493e+08,8.107532,53.968254
8,2030,600,150,0.90,8.0,4.375493e+08,9.729038,44.973545
9,2030,900,50,0.60,8.0,2.548081e+09,6.486025,392.857143


,Year,spread_€/t,ETS_price_€/tCO2,SAF_LCA_reduction,SAF_Share_%,Extra_cost_€,Avoided_CO2_Mt_LCA,Cost_per_tCO2_€/t
61,2050,600,150,0.75,80.0,6.010370e+09,111.368615,53.968254
62,2050,600,150,0.90,80.0,6.010370e+09,133.642338,44.973545
63,2050,900,50,0.60,80.0,3.500156e+10,89.094892,392.857143
64,2050,900,50,0.75,80.0,3.500156e+10,111.368615,314.285714
65,2050,900,50,0.90,80.0,3.500156e+10,133.642338,261.904762
66,2050,900,100,0.60,80.0,2.757699e+10,89.094892,309.523810
67,2050,900,100,0.75,80.0,2.757699e+10,111.368615,247.619048
68,2050,900,100,0.90,80.0,2.757699e+10,133.642338,206.349206
69,2050,900,150,0.60,80.0,2.015242e+10,89.094892,226.190476
70,2050,900,150,0.75,80.0,2.015242e+10,111.368615,180.952381


## 9. Cost model & sensitivity analysis

To analyse the **economic dimension** (Objective 2), we build a simple but interpretable cost model for **Scenario 1 (Policy)**.

### Assumptions:

- **Base jet fuel price**: `P_jet_base = 800 €/t`  
  → Reasonable mid-range based on recent jet fuel prices (IATA Fuel Price Monitor).
- **SAF price premium (`spread`)**: 600, 900, 1200 €/t  
  → Reflects SAF being ~2–5× fossil jet cost, especially for early generations and PtL SAF (ICCT, IATA, ICAO).
- **EU ETS carbon price (`ETS_price`)**: 50, 100, 150 €/tCO₂  
  → Consistent with recent and expected EU ETS price ranges under the Fit-for-55 package.
- **SAF lifecycle reduction (`SAF_LCA`)**: 0.6, 0.75, 0.9 (60%, 75%, 90%)  
  → Captures average vs best-in-class SAF pathways.

### Mechanics:

- Fossil jet **pays ETS**: effective price = `P_jet_base + EF_JET × ETS_price`
- SAF is assumed **exempt from ETS** (policy incentive)
- Compute:
  - Cost_baseline (all fossil + ETS)
  - Cost_with_SAF (mix of fossil + SAF)
  - Extra_cost_€ (incremental system cost)
  - Avoided_CO2_Mt_LCA (with lifecycle reductions)
  - **Cost_per_tCO2** = €/tCO₂ avoided

We loop over:

- SAF premium: **600 / 900 / 1200 €/t**
- ETS price: **50 / 100 / 150 €/tCO₂**
- SAF LCA: **60% / 75% / 90%**

and extract results for **2030, 2040, 2050**.

The outputs are stored in:

> `ie_iata_sensitivity_results.csv`

This sensitivity table is ideal for:

- Heatmaps of **€/tCO₂ avoided**
- Highlighting which policy levers (ETS price, reducing SAF premium, improving LCA) most improve cost-effectiveness.


In [22]:
import re
import numpy as np
import pandas as pd

EU27_ISO3 = [
    "AUT","BEL","BGR","HRV","CYP","CZE","DNK","EST","FIN","FRA",
    "DEU","GRC","HUN","IRL","ITA","LVA","LTU","LUX","MLT","NLD",
    "POL","PRT","ROU","SVK","SVN","ESP","SWE"
]

def clean_macro_table(path, year_min=1990, year_max=2050):
    """
    Cleaner for WB-style macro tables:
    - rows = countries, cols = years + metadata
    - keep EU27 only
    - extract proper year columns (1960–…)
    - aggregate to EU27
    - return Year x Value with NaN where all countries are missing
    """
    df = pd.read_csv(path)

    # --- 1. Detect country-code column ---
    country_col = None
    for cand in ["Country Code", "Country_Code", "CountryCode", "ISO3", "Code"]:
        if cand in df.columns:
            country_col = cand
            break

    if country_col is None:
        # fallback: choose column with most EU27 matches
        best_col, best_count = None, -1
        for col in df.columns:
            matches = df[col].isin(EU27_ISO3).sum()
            if matches > best_count:
                best_col, best_count = col, matches
        country_col = best_col

    df = df[df[country_col].isin(EU27_ISO3)].copy()

    # --- 2. Identify year columns (1960–2099 in the header) ---
    year_cols = []
    year_map = {}
    for col in df.columns:
        if col == country_col:
            continue
        m = re.search(r"(19[0-9]{2}|20[0-9]{2})", str(col))
        if m:
            y = int(m.group(1))
            year_cols.append(col)
            year_map[col] = y

    df_years = df[year_cols].copy()
    df_years.columns = [year_map[c] for c in df_years.columns]

    # --- 3. Transpose and aggregate EU27 ---
    df_years_T = df_years.T
    df_years_T.index.name = "Year"

    # min_count=1 → if all countries NaN → result NaN (not 0)
    df_out = pd.DataFrame(
        df_years_T.sum(axis=1, min_count=1),
        columns=["Value"]
    )

    # Optional: restrict to the years we care about
    df_out = df_out.loc[(df_out.index >= year_min) & (df_out.index <= year_max)]

    # Just to be safe: turn impossible “0 with all NaN” into NaN
    # (for variables that should never truly be 0 in EU27)
    # You can comment this out if you want to keep zeros.
    df_out["Value"] = df_out["Value"].replace({0.0: np.nan})

    return df_out.sort_index()


In [23]:
path_gdp  = raw_data_path / "gdb_wb.csv"
path_pop  = raw_data_path / "population_wb.csv"
path_air  = raw_data_path / "air transport_passenger_carried.csv"
path_tour = raw_data_path / "International tourism.csv"

df_gdp = clean_macro_table(path_gdp).rename(columns={"Value": "GDP_USD"})
df_pop = clean_macro_table(path_pop).rename(columns={"Value": "Population"})
df_air = clean_macro_table(path_air).rename(columns={"Value": "Air_Passengers"})
df_tour = clean_macro_table(path_tour).rename(columns={"Value": "Tourism_International"})

df_macro = (
    df_gdp
    .merge(df_pop, on="Year", how="outer")
    .merge(df_air, on="Year", how="outer")
    .merge(df_tour, on="Year", how="outer")
).sort_values("Year")

display(df_macro.head())
display(df_macro.tail())


,GDP_USD,Population,Air_Passengers,Tourism_International
Year,,,,
1990,6.491224e+12,420525030.0,157891000.0,NaN
1991,6.712667e+12,421741276.0,150528000.0,NaN
1992,7.372859e+12,422968989.0,164292000.0,NaN
1993,6.739804e+12,424344744.0,169958600.0,NaN
1994,7.146241e+12,425404352.0,182625300.0,NaN


,GDP_USD,Population,Air_Passengers,Tourism_International
Year,,,,
2020,1.550571e+13,446870959.0,2.245817e+08,332276000.0
2021,1.749852e+13,446227358.0,3.005104e+08,NaN
2022,1.699609e+13,447703403.0,NaN,NaN
2023,1.860267e+13,449425965.0,NaN,NaN
2024,1.942332e+13,450185396.0,NaN,NaN


In [41]:
df_cal = calibration.merge(df_macro, on="Year", how="left")

df_cal.to_csv("eu27_calibration_table_1990_2050_enriched.csv", index=False)


In [42]:
display(df_cal.head())
display(df_cal.tail())
df_cal.isna().sum()

,Year,Total_Fuel_Mt,CO2_Mt_state,TF,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg,CO2_Mt_calc,...,CO2_seg_regional,CO2_phase_climb,CO2_phase_cruise,CO2_phase_descent,CO2_phase_taxi-in,CO2_phase_taxi-out,GDP_USD,Population,Air_Passengers,Tourism_International
0,1990,22.607036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.212164,...,NaN,NaN,NaN,NaN,NaN,NaN,6.491224e+12,420525030.0,157891000.0,NaN
1,1991,22.050996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.460636,...,NaN,NaN,NaN,NaN,NaN,NaN,6.712667e+12,421741276.0,150528000.0,NaN
2,1992,23.298807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.391241,...,NaN,NaN,NaN,NaN,NaN,NaN,7.372859e+12,422968989.0,164292000.0,NaN
3,1993,24.054035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.770210,...,NaN,NaN,NaN,NaN,NaN,NaN,6.739804e+12,424344744.0,169958600.0,NaN
4,1994,25.042143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.882750,...,NaN,NaN,NaN,NaN,NaN,NaN,7.146241e+12,425404352.0,182625300.0,NaN


,Year,Total_Fuel_Mt,CO2_Mt_state,TF,CO2_TONS_g2g,NB_FLIGHTS_g2g,NOX_KG_g2g,SOX_KG_g2g,CO2_per_flight_kg_avg,CO2_Mt_calc,...,CO2_seg_regional,CO2_phase_climb,CO2_phase_cruise,CO2_phase_descent,CO2_phase_taxi-in,CO2_phase_taxi-out,GDP_USD,Population,Air_Passengers,Tourism_International
56,2046,55.300134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174.195421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,2047,56.184936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176.982548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2048,57.083895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179.814269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2049,57.997237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,182.691297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,2050,58.925193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.614358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Year                      0
Total_Fuel_Mt             0
CO2_Mt_state             45
TF                       45
CO2_TONS_g2g             55
NB_FLIGHTS_g2g           55
NOX_KG_g2g               55
SOX_KG_g2g               55
CO2_per_flight_kg_avg    55
CO2_Mt_calc               0
NB_FLIGHTS_g2g_full      55
CO2_seg_business         55
CO2_seg_cargo            55
CO2_seg_lowcost          55
CO2_seg_mainline         55
CO2_seg_other            55
CO2_seg_regional         55
CO2_phase_climb          55
CO2_phase_cruise         55
CO2_phase_descent        55
CO2_phase_taxi-in        55
CO2_phase_taxi-out       55
GDP_USD                  26
Population               26
Air_Passengers           29
Tourism_International    35
dtype: int64

### Final Calibration & Macro Table (1990–2050)

We build a single, consolidated EU27 dataset:

- **Fuel demand (Total_Fuel_Mt)**  
  Reconstructed from Eurostat "Final energy consumption – air transport" for EU27, converting TJ to Mt of jet fuel using an energy content of 0.043 TJ/kt.

- **CO₂ emissions and activity (EUROCONTROL)**  
  - `CO2_Mt_state`, `TF`: CO₂ and flights from the CO₂ by State dataset (EUROCONTROL ANS Performance).  
  - `CO2_TONS_g2g`, `NB_FLIGHTS_g2g`, `NOX_KG_g2g`, `SOX_KG_g2g`: gate-to-gate emissions and flights for EU27 departing flights.  
  - Segment-level CO₂ (`CO2_seg_*`) and flight-phase CO₂ (`CO2_phase_*`) from the G2G emissions breakdown.

- **Macroeconomic drivers (World Bank, EU aggregate)**  
  - `GDP_USD`: EU27 GDP (current US$), aggregated from World Bank country series.  
  - `Population`: EU27 population (total) aggregated from World Bank.  
  - `Air_Passengers`: air transport passengers carried (proxy for air travel demand), aggregated from World Bank; kept as NaN where no official data is available.

This table is **not** a submission file, but the internal backbone of the analysis.  
We use it to:
1. Check internal consistency between fuel-based CO₂, state-based CO₂, and G2G CO₂.  
2. Link aviation emissions to macro fundamentals (GDP, population, air travel).  
3. Feed macro-driven models for Scenario 0 (BAU) fuel demand and SAF adoption.


## Construct ML dataset

## 📌 Scenario Assumptions for EU27 (GDP, Population, Air Passengers, Flights)

To produce forward-looking fuel demand forecasts, we apply annual growth assumptions
to the key aviation and macroeconomic drivers. These assumptions are *not* official
forecasts but are grounded in long-term historical behavior of the EU27 economy and
aviation system. Sources are linked below.

---

### **1. GDP (current US$): +2% per year**
- Long-term real GDP growth in the EU has typically ranged **1–2%** annually.  
  **Source:** European Commission AMECO (Macro-Economic Database)  
  👉 https://economy-finance.ec.europa.eu/economic-research-and-databases/macro-economic-database-ameco_en
- ECB targets **~2% inflation**, which would imply nominal growth around 3–4%.  
  **Source:** European Central Bank — Inflation Target  
  👉 https://www.ecb.europa.eu/home/search/review/html/index.en.html
- To stay conservative and avoid FX distortions (USD-based GDP), we assume **2% nominal growth**.

---

### **2. Population: +0.1% per year**
- EU27 demographic projections show essentially **flat growth** (~0–0.2% annually).  
  **Source:** Eurostat population projections  
  👉 https://ec.europa.eu/eurostat/databrowser/view/PROJ_19NP/default/table?lang=en
- Aging and low fertility are offset by moderate net migration.
- We assume a small positive increment of **0.1% annually**.

---

### **3. Air Passengers: +3% per year**
- Pre-COVID European passenger traffic historically grew **3–5% per year**.  
  **Source:** Eurocontrol “European Aviation Outlook”  
  👉 https://www.eurocontrol.int/publication/european-aviation-outlook  
- Global air traffic typically grows faster than GDP (elasticity > 1).  
  **Source:** IATA Air Passenger Forecasts  
  👉 https://www.iata.org/en/publications/store/20-year-passenger-forecast/
- Medium-term stabilization suggests a realistic **3% annual passenger growth** assumption.

---

### **4. Number of Flights (NB_FLIGHTS_g2g): +2.5% per year**
- Flight movements grow more slowly than passengers because airlines increase load factors
  and aircraft size over time.  
  **Source:** Eurocontrol STATFOR Long-Term Forecast  
  👉 https://www.eurocontrol.int/publication/statfor-long-term-forecast  
- Typical long-run flight growth in Europe ranges **2–3%** annually.
- We assume **2.5% per year** as a conservative medium-term trend.

---

### **Summary of Scenario Growth Rates**
| Driver | Annual Growth Rate | Source |
|--------|---------------------|--------|
| **GDP_USD** | **+2.0%** | EC AMECO, ECB |
| **Population** | **+0.1%** | Eurostat |
| **Air_Passengers** | **+3.0%** | Eurocontrol, IATA |
| **NB_FLIGHTS_g2g** | **+2.5%** | Eurocontrol STATFOR |

---

These assumptions define a **Base Case scenario** for EU27 aviation fuel forecasting up to 2050.  
You can extend this into **Low** and **High** scenarios for sensitivity analysis (e.g., GDP 1% / 3%, air pax 2% / 4.5%, etc.).


In [43]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import joblib

# -----------------------------
# 1. CONFIG
# -----------------------------
ml_target = "Total_Fuel_Mt"
pred_col = "Fuel_S0_ML"  # final forecast: trend + residual

candidate_features = [
    'CO2_Mt_state', 'TF', 'CO2_TONS_g2g',
    'NB_FLIGHTS_g2g', 'NOX_KG_g2g', 'SOX_KG_g2g',
    'CO2_per_flight_kg_avg', 'CO2_Mt_calc', 'NB_FLIGHTS_g2g_full',
    'CO2_seg_business', 'CO2_seg_cargo', 'CO2_seg_lowcost',
    'CO2_seg_mainline', 'CO2_seg_other', 'CO2_seg_regional',
    'CO2_phase_climb', 'CO2_phase_cruise', 'CO2_phase_descent',
    'CO2_phase_taxi-in', 'CO2_phase_taxi-out',
    'GDP_USD', 'Population', 'Air_Passengers', 'Tourism_International'
]

growth_base_cols = [
    'GDP_USD', 'Population', 'Air_Passengers',
    'Tourism_International', 'NB_FLIGHTS_g2g', 'CO2_Mt_calc'
]

# Scenario growth assumptions (EU27 Base Case)
gdp_growth      = 0.02   # 2% per year
pop_growth      = 0.001  # 0.1% per year
air_growth      = 0.03   # 3% per year
flights_growth  = 0.025  # 2.5% per year

target_col = ml_target

# -----------------------------
# 2. FIT LINEAR TREND ON HISTORICAL FUEL
# -----------------------------
df_hist = df_cal.sort_values("Year").reset_index(drop=True)
df_hist_trend = df_hist[(~df_hist[ml_target].isna()) & (df_hist["Year"] < 2025)].copy()

trend_reg = LinearRegression()
trend_reg.fit(df_hist_trend[["Year"]], df_hist_trend[ml_target])

print("Trend slope (Mt per year):", trend_reg.coef_[0])
print("Trend intercept:", trend_reg.intercept_)

# -----------------------------
# 3. BUILD TRAINING DATA (df_ml) WITH RESIDUALS
# -----------------------------
df_ml = df_cal.sort_values("Year").reset_index(drop=True)
df_ml["t"] = np.arange(len(df_ml))

# compute trend for all rows
df_ml["Fuel_trend"] = trend_reg.predict(df_ml[["Year"]])

# residual = actual - trend (only meaningful where target is observed)
df_ml["Fuel_resid"] = df_ml[ml_target] - df_ml["Fuel_trend"]

# growths
for col in growth_base_cols:
    if col in df_ml.columns:
        df_ml[f"{col}_growth"] = df_ml[col].pct_change()

# lags of TOTAL fuel (not residual)
df_ml[f"{target_col}_lag1"] = df_ml[ml_target].shift(1)
df_ml[f"{target_col}_lag2"] = df_ml[ml_target].shift(2)

# restrict to years < 2025 and drop rows without lags or residuals
df_ml = df_ml[df_ml["Year"] < 2025].dropna(subset=[f"{target_col}_lag1", f"{target_col}_lag2", "Fuel_resid"])

# final feature list for residual model
ml_features = (
    ["Year", "t"] +
    candidate_features +
    [f"{col}_growth" for col in growth_base_cols if f"{col}_growth" in df_ml.columns] +
    [f"{target_col}_lag1", f"{target_col}_lag2"]
)

X = df_ml[ml_features].values
y_resid = df_ml["Fuel_resid"].values  # <-- model learns residuals, not raw fuel

print("Features used:")
print(ml_features)
print("\nTraining data shape:", X.shape, "residual target shape:", y_resid.shape)

# -----------------------------
# 4. TRAIN / TEST SPLIT
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_resid, test_size=0.2, shuffle=False
)

# -----------------------------
# 5. PIPELINE + GRID SEARCH (RESIDUAL MODEL)
# -----------------------------
full_pipeline = Pipeline([
    ("imputer", KNNImputer()),
    ("model", HistGradientBoostingRegressor(
        random_state=42,
        loss="squared_error"
    ))
])

param_grid = {
    "imputer__n_neighbors": [3],
    "model__max_depth": [3, 5],
    "model__learning_rate": [0.05, 0.1],
    "model__max_iter": [200, 300],
    "model__min_samples_leaf": [10, 20],
}

tscv = TimeSeriesSplit(n_splits=3)

grid_search = GridSearchCV(
    full_pipeline,
    param_grid,
    cv=tscv,
    n_jobs=-1,
    verbose=1,
    scoring="neg_mean_squared_error"
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

print("\nBest params (residual model):", grid_search.best_params_)

# -----------------------------
# 6. METRICS ON RESIDUALS & FULL FUEL
# -----------------------------
# residual predictions
y_pred_train_resid = best_model.predict(X_train)
y_pred_test_resid  = best_model.predict(X_test)

# reconstruct full fuel on train/test slices of df_ml
df_ml_train = df_ml.iloc[:len(X_train)].copy()
df_ml_test  = df_ml.iloc[len(X_train):len(X_train)+len(X_test)].copy()

df_ml_train["Fuel_resid_pred"] = y_pred_train_resid
df_ml_test["Fuel_resid_pred"]  = y_pred_test_resid

df_ml_train["Fuel_ML"] = df_ml_train["Fuel_trend"] + df_ml_train["Fuel_resid_pred"]
df_ml_test["Fuel_ML"]  = df_ml_test["Fuel_trend"] + df_ml_test["Fuel_resid_pred"]

metrics = pd.DataFrame({
    "Set": ["Train", "Test"],
    "R2": [
        r2_score(df_ml_train[ml_target], df_ml_train["Fuel_ML"]),
        r2_score(df_ml_test[ml_target],  df_ml_test["Fuel_ML"])
    ],
    "MAE": [
        mean_absolute_error(df_ml_train[ml_target], df_ml_train["Fuel_ML"]),
        mean_absolute_error(df_ml_test[ml_target],  df_ml_test["Fuel_ML"])
    ],
    "RMSE": [
        np.sqrt(mean_squared_error(df_ml_train[ml_target], df_ml_train["Fuel_ML"])),
        np.sqrt(mean_squared_error(df_ml_test[ml_target],  df_ml_test["Fuel_ML"]))
    ]
})

print("\n--- Model Performance (Trend + Residual ML) ---")
display(metrics)

# -----------------------------
# 7. SAVE RESIDUAL MODEL + TREND
# -----------------------------
joblib.dump(best_model, "model_residual_boosted.pkl")
joblib.dump(trend_reg, "model_trend_linear.pkl")
print("✅ models saved as model_residual_boosted.pkl and model_trend_linear.pkl")

# -----------------------------
# 8. RECURSIVE FORECAST 2025–2050 WITH TREND + RESIDUALS
# -----------------------------
df_full = df_cal.sort_values("Year").reset_index(drop=True)

# identify last historical year where target is observed
last_hist_year = df_hist_trend["Year"].max()

# base values at last historical year for scenario growth
base_row = df_full[df_full["Year"] == last_hist_year].iloc[0]

base_gdp     = base_row["GDP_USD"]
base_pop     = base_row["Population"]
base_air     = base_row["Air_Passengers"]
base_flights = base_row["NB_FLIGHTS_g2g"] if "NB_FLIGHTS_g2g" in df_full.columns else np.nan

# apply growth scenario for future years
years_ahead = df_full["Year"] - last_hist_year
future_mask_all = df_full["Year"] > last_hist_year

df_full.loc[future_mask_all, "GDP_USD"] = base_gdp * (1 + gdp_growth) ** years_ahead[future_mask_all]
df_full.loc[future_mask_all, "Population"] = base_pop * (1 + pop_growth) ** years_ahead[future_mask_all]
df_full.loc[future_mask_all, "Air_Passengers"] = base_air * (1 + air_growth) ** years_ahead[future_mask_all]

if "NB_FLIGHTS_g2g" in df_full.columns and not np.isnan(base_flights):
    df_full.loc[future_mask_all, "NB_FLIGHTS_g2g"] = base_flights * (1 + flights_growth) ** years_ahead[future_mask_all]

# rebuild features on full horizon
df_full["t"] = np.arange(len(df_full))

for col in growth_base_cols:
    if col in df_full.columns:
        df_full[f"{col}_growth"] = df_full[col].pct_change()

# compute trend for full horizon
df_full["Fuel_trend"] = trend_reg.predict(df_full[["Year"]])

# lags of TOTAL fuel (we'll fill future ones recursively)
df_full[f"{target_col}_lag1"] = df_full[ml_target].shift(1)
df_full[f"{target_col}_lag2"] = df_full[ml_target].shift(2)

# column for residual predictions & final forecasts
df_full["Fuel_resid_pred"] = np.nan
df_full[pred_col] = np.nan

future_mask = df_full["Year"].between(2025, 2050)
future_idx = df_full.index[future_mask]

for idx in future_idx:
    pos = df_full.index.get_loc(idx)

    # fill lag1, lag2 using previous TOTAL FUEL (actual or predicted)
    for lag_k in (1, 2):
        lag_col = f"{target_col}_lag{lag_k}"
        if pd.isna(df_full.loc[idx, lag_col]) and pos - lag_k >= 0:
            prev_idx = df_full.index[pos - lag_k]
            prev_val = df_full.loc[prev_idx, ml_target]
            if pd.isna(prev_val):
                prev_val = df_full.loc[prev_idx, pred_col]
            df_full.loc[idx, lag_col] = prev_val

    # build feature row
    X_row = df_full.loc[idx, ml_features].to_frame().T

    # predict residual
    resid_hat = best_model.predict(X_row)[0]
    df_full.loc[idx, "Fuel_resid_pred"] = resid_hat

    # compute final fuel = trend + residual
    trend_val = df_full.loc[idx, "Fuel_trend"]
    fuel_hat = trend_val + resid_hat
    df_full.loc[idx, pred_col] = fuel_hat

    # treat prediction as observed target for future lags
    if pd.isna(df_full.loc[idx, ml_target]):
        df_full.loc[idx, ml_target] = fuel_hat

# final future df
df_future = df_full[future_mask].copy()
print("\n--- Forecast 2025–2050 (Trend + Residual ML, Base Scenario) ---")
display(df_future[["Year", pred_col]])



Trend slope (Mt per year): 0.4543364975050484
Trend intercept: -878.126749177682
Features used:
['Year', 't', 'CO2_Mt_state', 'TF', 'CO2_TONS_g2g', 'NB_FLIGHTS_g2g', 'NOX_KG_g2g', 'SOX_KG_g2g', 'CO2_per_flight_kg_avg', 'CO2_Mt_calc', 'NB_FLIGHTS_g2g_full', 'CO2_seg_business', 'CO2_seg_cargo', 'CO2_seg_lowcost', 'CO2_seg_mainline', 'CO2_seg_other', 'CO2_seg_regional', 'CO2_phase_climb', 'CO2_phase_cruise', 'CO2_phase_descent', 'CO2_phase_taxi-in', 'CO2_phase_taxi-out', 'GDP_USD', 'Population', 'Air_Passengers', 'Tourism_International', 'GDP_USD_growth', 'Population_growth', 'Air_Passengers_growth', 'Tourism_International_growth', 'NB_FLIGHTS_g2g_growth', 'CO2_Mt_calc_growth', 'Total_Fuel_Mt_lag1', 'Total_Fuel_Mt_lag2']

Training data shape: (33, 34) residual target shape: (33,)
Fitting 3 folds for each of 16 candidates, totalling 48 fits


C:\Users\Radwan\AppData\Local\Temp\ipykernel_14856\2992051647.py:69: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_ml[f"{col}_growth"] = df_ml[col].pct_change()
C:\Users\Radwan\AppData\Local\Temp\ipykernel_14856\2992051647.py:69: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_ml[f"{col}_growth"] = df_ml[col].pct_change()
C:\Users\Radwan\AppData\Local\Temp\ipykernel_14856\2992051647.py:69: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fil


Best params (residual model): {'imputer__n_neighbors': 3, 'model__learning_rate': 0.05, 'model__max_depth': 3, 'model__max_iter': 200, 'model__min_samples_leaf': 20}

--- Model Performance (Trend + Residual ML) ---


,Set,R2,MAE,RMSE
0,Train,0.783487,1.986846,2.529153
1,Test,-0.300760,8.392230,10.618764


✅ models saved as model_residual_boosted.pkl and model_trend_linear.pkl

--- Forecast 2025–2050 (Trend + Residual ML, Base Scenario) ---


C:\Users\Radwan\AppData\Local\Temp\ipykernel_14856\2992051647.py:210: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_full[f"{col}_growth"] = df_full[col].pct_change()
C:\Users\Radwan\AppData\Local\Temp\ipykernel_14856\2992051647.py:210: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_full[f"{col}_growth"] = df_full[col].pct_change()
c:\Users\Radwan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  warnings.warn(
c:\Users\Radwan\AppData\Local\Programs\Python\Python312\L

,Year,Fuel_S0_ML
35,2025,43.154258
36,2026,43.608595
37,2027,44.062931
38,2028,44.517268
39,2029,44.971604
40,2030,45.425941
41,2031,45.880277
42,2032,46.334614
43,2033,46.788950
44,2034,47.243287
